# Lezione 22 - Clustering Gerarchico

## Sezione 1 - Titolo e obiettivi

---

## Mappa della lezione

| Sezione | Contenuto | Tempo stimato |
|---------|-----------|---------------|
| 1 | Titolo, obiettivi, confronto con K-Means | 5 min |
| 2 | Teoria profonda: agglomerativo, linkage, dendrogrammi | 20 min |
| 3 | Schema mentale: workflow gerarchico | 5 min |
| 4 | Demo: costruzione e lettura dendrogramma | 20 min |
| 5 | Esercizi risolti + errori comuni | 20 min |
| 6 | Conclusione operativa | 10 min |
| 7 | Checklist di fine lezione + glossario | 5 min |
| 8 | Changelog didattico | 2 min |

---

## Obiettivi della lezione

Al termine di questa lezione sarai in grado di:

| # | Obiettivo | Verifica |
|---|-----------|----------|
| 1 | Costruire e interpretare un **dendrogramma** | Sai leggere le "gambe lunghe"? |
| 2 | Differenziare i metodi di **linkage** | Sai quando Ward vs Single? |
| 3 | **Tagliare** il dendrogramma per ottenere K cluster | Sai usare fcluster? |
| 4 | Valutare i cluster con **silhouette** | Conosci la procedura? |
| 5 | Confrontare gerarchico vs K-Means | Sai quando usare l'uno o l'altro? |

---

## Il vantaggio chiave: K non serve a priori

```
K-MEANS:                         GERARCHICO:
   │                                │
   ▼                                ▼
"Dammi K cluster"               "Ti mostro TUTTI i livelli"
   │                                │
   ▼                                ▼
Devi decidere K prima           Decidi K dopo, guardando il dendrogramma
```

**Quando usare gerarchico:**
- Non sai quanti cluster ci sono
- Vuoi vedere la struttura multi-livello
- Dataset piccolo/medio (< 10k punti)
- Esiste una gerarchia naturale nei dati

---

## I 4 metodi di linkage (sintesi visiva)

```
SINGLE (min):     COMPLETE (max):    AVERAGE:         WARD:
   A    B            A    B          A    B           A    B
   ●    ●            ●    ●          ●    ●           ●    ●
   │    │            │    │          │    │         var(A∪B)
   ●────●            ●    ●          ●────●            ↓
   min dist          │    │          all pairs      minimize
                     ●────●                         Δ variance
                     max dist
```

| Linkage | Default? | Pro | Contro |
|---------|----------|-----|--------|
| **Ward** | ✅ Consigliato | Cluster bilanciati, simile a K-Means | Assume sfericità |
| Complete | No | Cluster compatti | Sensibile a outlier |
| Average | No | Compromesso | Meno intuitivo |
| Single | ⚠️ Da evitare | Trova forme allungate | Effetto catena |

---

## Prerequisiti minimi

| Concetto | Dove lo trovi | Verifica |
|----------|---------------|----------|
| K-Means | Lezione 20 | Conosci inertia e centroidi? |
| Scelta K | Lezione 21 | Sai usare Elbow e Silhouette? |
| StandardScaler | Lezione 13, 20 | Sai perché scalare? |

**Micro-checkpoint prerequisiti:**
```python
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.cluster import AgglomerativeClustering
print("OK!" if callable(linkage) else "Installa scipy")
```

## Sezione 2 - Teoria profonda

### 1.1 Clustering gerarchico: agglomerativo vs divisivo
- Agglomerativo (bottom-up): ogni punto e un cluster, poi unisci i due cluster piu simili fino a ottenerne uno solo. E l'approccio usato da sklearn.
- Divisivo (top-down): parti da un unico cluster con tutti i punti e lo dividi iterativamente. Meno comune, non implementato in sklearn.

### 1.2 Dendrogramma: la mappa delle fusioni
- Asse Y: distanza/dissimilarita alla quale avviene l'unione.
- Linee orizzontali: fusioni; piu sono alte, piu i cluster erano lontani.
- Taglio orizzontale: determina il numero di cluster. Taglio alto = pochi cluster generali; taglio basso = molti cluster specifici.


### 1.3 Metodi di linkage (distanza tra cluster)

| Linkage | Distanza tra cluster | Pro | Contro |
|---------|---------------------|-----|--------|
| Single | Distanza minima tra coppie | Rileva forme allungate | Effetto catena (unisce cluster lontani tramite punti ponte) |
| Complete | Distanza massima tra coppie | Cluster compatti | Sensibile a outlier |
| Average | Media di tutte le coppie | Compromesso | Piu costoso da intuire |
| Ward | Aumento di varianza totale | Cluster bilanciati, simile a K-Means | Assume forma quasi sferica |

Formule sintetiche (A, B cluster):
- Single: $d=\min_{a\in A, b\in B} d(a,b)$
- Complete: $d=\max_{a\in A, b\in B} d(a,b)$
- Average: media di tutte le distanze tra i punti dei due cluster
- Ward: minimizza l'incremento di varianza intra-cluster ($\propto \|ar a-ar b\|$)

Regola pratica: prova prima Ward; passa a complete/average se i cluster non sono sferici; evita single se sospetti l'effetto catena.


### 1.4 Confronto gerarchico vs K-Means

| Aspetto | K-Means | Gerarchico |
|---------|---------|------------|
| K richiesto a priori | Si | No (lo scegli a posteriori) |
| Complessita | O(n*K*iter) | O(n^2) o O(n^3) (meno scalabile) |
| Forma cluster | Sferica | Dipende dal linkage |
| Determinismo | No (init random) | Si (dato il linkage) |
| Output | Etichette e centroidi | Albero (dendrogramma) e etichette |

Quando usare gerarchico: dataset piccoli/medi (<10k), vuoi piu livelli di granularita, o esiste una gerarchia naturale. Quando usare K-Means: dataset grandi, hai un K ipotizzato, vuoi velocita.


### 1.5 Come leggere e tagliare un dendrogramma

1) Osserva le altezze delle fusioni: gambe lunghe suggeriscono separazioni naturali.
2) Decidi un'altezza di taglio: tagliando la linea orizzontale conti quante fusioni attraversi = numero di cluster.
3) Valida con silhouette score e con il contesto (cluster interpretabili?).

Taglio troppo alto -> pochi cluster (potresti perdere dettaglio). Taglio troppo basso -> molti cluster (rischio over-segmentazione).


### 1.6 Gerarchia e silhouette

Silhouette resta utile anche nel gerarchico:
- Silhouette globale: qualita media della separazione tra cluster.
- Silhouette plot per cluster: individua cluster sottili o con valori negativi.

Usa silhouette per confrontare altezze di taglio (o valori di K) e scegliere il compromesso migliore tra separazione e interpretabilita.


## Sezione 3 - Schema mentale e decision map

Workflow sintetico dopo lo scaling:

```
DATI SCALATI
   |
   +-- Calcola linkage (scipy.linkage) con metodo scelto
   +-- Visualizza dendrogramma
   +-- Scegli altezza di taglio (gambe lunghe, salti nelle fusioni)
   +-- Estrai cluster (fcluster) o usa AgglomerativeClustering con K
   +-- Valuta con silhouette e con il contesto
   +-- Interpreta i cluster
```

Checklist rapida
- [ ] Scaling applicato (StandardScaler).
- [ ] Metodo di linkage scelto e motivato.
- [ ] Dendrogramma letto (gambe lunghe e salti nelle fusioni).
- [ ] Altezza di taglio decisa e numero di cluster verificato.
- [ ] Silhouette calcolata e controllata per cluster deboli.
- [ ] Cluster interpretati e rinominati in modo leggibile.


## Sezione 4 - Notebook dimostrativo

### Perche questo passo (Demo 1 - Primo dendrogramma)
Costruiamo il primo dendrogramma per capire cosa rappresentano distanze e taglio. Ci aspettiamo 3 cluster evidenti e gambe lunghe coerenti con i cluster generati.


In [ ]:
# Demo 1: Primo dendrogramma su dati sintetici (attesi 3 cluster)
# Perche: vedere come il dendrogramma riflette le fusioni e dove tagliare.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
from scipy.cluster.hierarchy import dendrogram, linkage

np.random.seed(42)
X, y_true = make_blobs(n_samples=30, centers=3, cluster_std=0.8, random_state=42)
assert X.ndim == 2, "Atteso array 2D"
assert not np.isnan(X).any(), "Dati con NaN"

# Scaling: rende le feature confrontabili
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
assert X_scaled.shape == X.shape, "Shape inattesa dopo scaling"

# Calcolo linkage (Ward)
Z = linkage(X_scaled, method='ward')
assert Z.shape[0] == X_scaled.shape[0] - 1, "Linkage deve avere n-1 fusioni"

print(f"Dataset: {X.shape[0]} punti, {X.shape[1]} feature")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].scatter(X_scaled[:, 0], X_scaled[:, 1], c=y_true, cmap='viridis', s=80, edgecolors='black')
axes[0].set_title('Dati scalati con cluster noti=3')
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')
axes[0].grid(True, alpha=0.3)

# Dendrogramma con linea di taglio suggerita
line = dendrogram(Z, ax=axes[1], leaf_rotation=90, leaf_font_size=8)
axes[1].axhline(y=5, color='red', linestyle='--', linewidth=2, label='Taglio suggerito')
axes[1].set_xlabel('Indice punto (riordinato)')
axes[1].set_ylabel('Distanza (Ward)')
axes[1].set_title('Dendrogramma (Ward)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Check atteso: gambe lunghe prima del taglio a 5 e circa 3 cluster


### Perche questo passo (Demo 2 - Confronto linkage)
Confrontiamo single, complete, average, Ward sullo stesso dataset. Obiettivo: vedere differenze visive nei dendrogrammi e nei silhouette, e capire quando un metodo e preferibile.


In [ ]:
# Demo 2: Confronto dei metodi di linkage sullo stesso dataset
# Perche: mostrare differenze visive e di silhouette tra single/complete/average/Ward.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram

assert 'X_scaled' in globals(), "Esegui prima Demo 1 per creare X_scaled"

linkage_methods = ['single', 'complete', 'average', 'ward']
fig, axes = plt.subplots(2, 4, figsize=(18, 9))

for idx, method in enumerate(linkage_methods):
    Z = linkage(X_scaled, method=method)
    labels = fcluster(Z, t=3, criterion='maxclust')
    sil = silhouette_score(X_scaled, labels)
    dendrogram(Z, ax=axes[0, idx], leaf_rotation=90, leaf_font_size=6, truncate_mode='lastp', p=12)
    axes[0, idx].set_title(f'{method.upper()} linkage')
    axes[0, idx].set_xlabel('Cluster')
    axes[0, idx].set_ylabel('Distanza')
    axes[0, idx].grid(True, alpha=0.3)
    axes[1, idx].scatter(X_scaled[:, 0], X_scaled[:, 1], c=labels, cmap='viridis', s=70, edgecolors='black', alpha=0.8)
    axes[1, idx].set_title(f'Silhouette={sil:.3f}')
    axes[1, idx].set_xlabel('Feature 1')
    axes[1, idx].set_ylabel('Feature 2')
    axes[1, idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

for method in linkage_methods:
    Z = linkage(X_scaled, method=method)
    labels = fcluster(Z, t=3, criterion='maxclust')
    sil = silhouette_score(X_scaled, labels)
    note = {
        'single': 'Rischio catena',
        'complete': 'Cluster compatti',
        'average': 'Compromesso',
        'ward': 'Simile a K-Means'
    }[method]
    print(f"{method:<10} silhouette={sil:.3f} note={note}")

# Check: atteso Ward con silhouette piu alta su cluster sferici


### Perche questo passo (Demo 3 - Effetto catena)
Mostriamo il limite del single linkage quando esiste un ponte di punti tra cluster. Obiettivo: riconoscere il sintomo (silhouette basso, cluster allungati) e preferire Ward/complete.


In [ ]:
# Demo 3: Effetto catena del single linkage
# Perché: mostrare come punti ponte possano far collassare due cluster separati con single linkage.

import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

np.random.seed(42)

cluster1 = np.random.randn(30, 2) + np.array([-3, 0])
cluster2 = np.random.randn(30, 2) + np.array([3, 0])
ponte = np.array([[-2, 0], [-1, 0], [0, 0], [1, 0], [2, 0]])

X_chain = np.vstack([cluster1, cluster2, ponte])
X_chain_scaled = StandardScaler().fit_transform(X_chain)

# Sanity check
assert not np.isnan(X_chain_scaled).any(), "NaN nei dati"

fig, axes = plt.subplots(2, 3, figsize=(15, 9))

for idx, method in enumerate(["single", "ward"]):
    Z = linkage(X_chain_scaled, method=method)

    labels = fcluster(Z, t=2, criterion="maxclust")
    sil = silhouette_score(X_chain_scaled, labels)

    dendrogram(
        Z,
        ax=axes[0, idx],
        truncate_mode="lastp",
        p=20,
        leaf_rotation=90,
        leaf_font_size=7
    )
    axes[0, idx].set_title(f"Dendrogramma {method}")
    axes[0, idx].set_ylabel("Distanza")
    axes[0, idx].grid(True, alpha=0.3)

    axes[1, idx].scatter(
        X_chain_scaled[:, 0],
        X_chain_scaled[:, 1],
        c=labels,
        cmap="viridis",
        s=60,
        edgecolors="black",
        alpha=0.7
    )

    axes[1, idx].scatter(
        X_chain_scaled[-5:, 0],
        X_chain_scaled[-5:, 1],
        c="orange",
        s=140,
        marker="s",
        edgecolors="black",
        label="Punti ponte"
    )

    axes[1, idx].set_title(f"{method.upper()} | silhouette = {sil:.3f}")
    axes[1, idx].grid(True, alpha=0.3)
    axes[1, idx].legend()

axes[0, 2].axis("off")
axes[1, 2].axis("off")

axes[1, 2].text(
    0.5,
    0.5,
    (
        "Single linkage usa la distanza minima.\n\n"
        "Una catena di pochi punti ponte\n"
        "può unire cluster lontani.\n\n"
        "Ward minimizza la varianza interna,\n"
        "quindi resiste all'effetto catena."
    ),
    transform=axes[1, 2].transAxes,
    fontsize=11,
    ha="center",
    va="center",
    bbox=dict(boxstyle="round", facecolor="lightyellow", alpha=0.8)
)

plt.tight_layout()
plt.show()

print(
    "Conclusione: se esistono punti ponte, "
    "il single linkage produce cluster allungati; "
    "Ward (o complete) è spesso preferibile."
)


### Perche questo passo (Demo 4 - Taglio del dendrogramma)
Mostriamo strategie pratiche per scegliere l'altezza di taglio: gambe lunghe, salti nelle distanze, confronto silhouette.


In [ ]:
# Demo 4: Taglio del dendrogramma e scelta di K
# Perche: mostrare metodi pratici (gambe lunghe e salti) per decidere K.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram

np.random.seed(42)
X_multi, _ = make_blobs(n_samples=100, centers=4, cluster_std=[0.8, 1.0, 0.6, 1.2], center_box=(-10, 10), random_state=42)
X_multi_scaled = StandardScaler().fit_transform(X_multi)
assert not np.isnan(X_multi_scaled).any()

Z = linkage(X_multi_scaled, method='ward')

fusion_dist = Z[:, 2]
jumps = np.diff(fusion_dist)
max_jump_idx = int(np.argmax(jumps))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
dendrogram(Z, ax=axes[0], truncate_mode='lastp', p=25, leaf_rotation=90, leaf_font_size=7)
axes[0].axhline(y=5, color='red', linestyle='--', label='Taglio esempio')
axes[0].set_title('Dendrogramma (Ward)')
axes[0].set_xlabel('Cluster')
axes[0].set_ylabel('Distanza')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].bar(range(len(jumps)), jumps, color='steelblue', alpha=0.7)
axes[1].bar(max_jump_idx, jumps[max_jump_idx], color='red', alpha=0.8, label='Salto massimo')
axes[1].set_title('Salti tra fusioni (aiuta a scegliere K)')
axes[1].set_xlabel('Indice fusione')
axes[1].set_ylabel('Delta distanza')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

labels = fcluster(Z, t=4, criterion='maxclust')
sil = silhouette_score(X_multi_scaled, labels)
print(f"Taglio a 4 cluster -> silhouette={sil:.3f}")


### Perche questo passo (Demo 5 - Caso ambiguo)
Dataset con cluster sovrapposti: nessun metodo fornisce un K univoco. Obiettivo: usare silhouette plot e principio di parsimonia (scegliere il K piu semplice e interpretabile).


In [ ]:
# Demo 5: Caso ambiguo con cluster sovrapposti
# Perche: vedere cosa fare quando silhouette non mostra un picco chiaro.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import linkage, dendrogram

np.random.seed(42)
X_hard, _ = make_blobs(n_samples=300, centers=5, cluster_std=1.8, random_state=42)
X_hard_scaled = StandardScaler().fit_transform(X_hard)
assert not np.isnan(X_hard_scaled).any()

K_range_sil = range(2, 11)
sil_scores_hard = []
for k in K_range_sil:
    km = AgglomerativeClustering(n_clusters=k, linkage='ward')
    labels = km.fit_predict(X_hard_scaled)
    sil_scores_hard.append(silhouette_score(X_hard_scaled, labels))

fig, axes = plt.subplots(2, 3, figsize=(15, 9))
axes[0, 0].plot(K_range_sil, sil_scores_hard, 'go-')
axes[0, 0].set_title('Silhouette (caso ambiguo)')
axes[0, 0].set_xlabel('K')
axes[0, 0].set_ylabel('Silhouette')
axes[0, 0].axhline(y=0.25, color='orange', linestyle=':', alpha=0.6)
axes[0, 0].grid(True, alpha=0.3)

Z_hard = linkage(X_hard_scaled, method='ward')
dendrogram(Z_hard, ax=axes[0,1], truncate_mode='lastp', p=25, leaf_rotation=90, leaf_font_size=7)
axes[0,1].set_title('Dendrogramma (caso ambiguo)')
axes[0,1].set_xlabel('Cluster')
axes[0,1].set_ylabel('Distanza')
axes[0,1].grid(True, alpha=0.3)
axes[0,2].axis('off')

for idx, k in enumerate([3,4,5]):
    km = AgglomerativeClustering(n_clusters=k, linkage='ward')
    labels = km.fit_predict(X_hard_scaled)
    sil_vals = silhouette_samples(X_hard_scaled, labels)
    y_lower = 10
    colors = plt.cm.viridis(np.linspace(0,1,k))
    ax = axes[1, idx]
    for i in range(k):
        vals = sil_vals[labels == i]
        vals.sort()
        size = len(vals)
        y_upper = y_lower + size
        ax.fill_betweenx(np.arange(y_lower, y_upper), 0, vals, facecolor=colors[i], edgecolor=colors[i], alpha=0.7)
        ax.text(-0.05, y_lower + size/2, f'C{i}')
        y_lower = y_upper + 10
    ax.axvline(x=np.mean(sil_vals), color='red', linestyle='--')
    ax.set_title(f'Silhouette plot K={k}')
    ax.set_xlim([-0.1,1])
    ax.set_xlabel('Silhouette score')
    ax.set_ylabel('Cluster')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Se i metodi non concordano, preferisci il K piu parsimonioso e interpretabile (es. 3).")


## Sezione 5 - Esercizi guidati (step by step)

### Perche questo esercizio (22.1)
Eseguire un'analisi completa con dendrogramma, scelta di K, silhouette e interpretazione su dati medici. Obiettivo: applicare il flusso end-to-end.


In [ ]:
# Esercizio 22.1 - Analisi completa con dendrogramma
# Obiettivo: seguire l'intero flusso (dendrogramma -> scelta K -> silhouette -> profili).

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram

eta = [25, 30, 28, 65, 70, 68, 45, 48, 50, 22, 75, 42]
pressione = [120, 125, 118, 145, 155, 150, 130, 135, 132, 115, 160, 128]
colesterolo = [180, 190, 175, 240, 260, 250, 210, 215, 205, 170, 270, 200]

df_pazienti = pd.DataFrame({
    'paziente': [f'P{i}' for i in range(1, 13)],
    'eta': eta,
    'pressione': pressione,
    'colesterolo': colesterolo
})
print("Dataset pazienti (head):")
print(df_pazienti.head())

X_pazienti = df_pazienti[['eta', 'pressione', 'colesterolo']].values
scaler = StandardScaler()
X_pazienti_scaled = scaler.fit_transform(X_pazienti)
assert X_pazienti_scaled.shape == X_pazienti.shape
assert not np.isnan(X_pazienti_scaled).any()

Z = linkage(X_pazienti_scaled, method='ward')

fusion_dist = Z[:,2]
jumps = np.diff(fusion_dist)
max_jump_idx = int(np.argmax(jumps))
K_suggerito = X_pazienti_scaled.shape[0] - max_jump_idx - 1

fig, axes = plt.subplots(1, 2, figsize=(14,5))
dendrogram(Z, ax=axes[0], labels=df_pazienti['paziente'].values, leaf_rotation=45)
axes[0].axhline(y=fusion_dist[max_jump_idx], color='red', linestyle='--', label=f'K stimato ~{K_suggerito}')
axes[0].set_title('Dendrogramma pazienti (Ward)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[1].bar(range(len(jumps)), jumps, color='steelblue')
axes[1].bar(max_jump_idx, jumps[max_jump_idx], color='red', label='Salto massimo')
axes[1].set_title('Salti nelle distanze')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

K_ottimale = 3
labels = fcluster(Z, t=K_ottimale, criterion='maxclust') - 1
sil = silhouette_score(X_pazienti_scaled, labels)
print(f"K scelto: {K_ottimale}, silhouette={sil:.3f}")

df_pazienti['cluster'] = labels
print("Profili medi per cluster:")
print(df_pazienti.groupby('cluster')[['eta','pressione','colesterolo']].mean().round(1))

fig, ax = plt.subplots(figsize=(8,6))
scatter = ax.scatter(df_pazienti['eta'], df_pazienti['colesterolo'], c=df_pazienti['cluster'], cmap='viridis', s=100, edgecolors='black')
for _, row in df_pazienti.iterrows():
    ax.annotate(row['paziente'], (row['eta']+1, row['colesterolo']+3), fontsize=8)
ax.set_xlabel('Eta')
ax.set_ylabel('Colesterolo')
ax.set_title(f'Clustering pazienti (K={K_ottimale}, silhouette={sil:.3f})')
ax.grid(True, alpha=0.3)
plt.colorbar(scatter, label='Cluster')
plt.tight_layout()
plt.show()


### Perche questo esercizio (22.2)
Confrontare i metodi di linkage su cluster di forme diverse. Obiettivo: scegliere il metodo migliore motivando con silhouette e struttura.


In [ ]:
# Esercizio 22.2 - Confronto metodi di linkage su forme diverse
# Obiettivo: vedere quale linkage gestisce meglio cluster ellittici/compatti/sparsi.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, adjusted_rand_score
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram

np.random.seed(42)
n1, n2, n3 = 20, 20, 20
cluster1 = np.column_stack([
    np.random.normal(0, 0.5, n1),
    np.random.normal(0, 2.5, n1)
])
cluster2 = np.random.normal(loc=[5, 0], scale=0.5, size=(n2, 2))
cluster3 = np.random.normal(loc=[2.5, 5], scale=1.5, size=(n3, 2))
X_forme = np.vstack([cluster1, cluster2, cluster3])
y_true = np.array([0]*n1 + [1]*n2 + [2]*n3)
assert X_forme.shape[0] == 60

X_forme_scaled = StandardScaler().fit_transform(X_forme)

metodi = ['single', 'complete', 'average', 'ward']
fig, axes = plt.subplots(2, 4, figsize=(17, 8))
risultati = {}

for idx, metodo in enumerate(metodi):
    Z = linkage(X_forme_scaled, method=metodo)
    labels = fcluster(Z, t=3, criterion='maxclust') - 1
    sil = silhouette_score(X_forme_scaled, labels)
    ari = adjusted_rand_score(y_true, labels)
    risultati[metodo] = {'sil': sil, 'ari': ari}
    dendrogram(Z, ax=axes[0, idx], truncate_mode='lastp', p=10, leaf_rotation=45, no_labels=True)
    axes[0, idx].set_title(metodo.upper())
    axes[0, idx].grid(True, alpha=0.3)
    axes[1, idx].scatter(X_forme[:, 0], X_forme[:, 1], c=labels, cmap='viridis', s=50, edgecolors='black', alpha=0.8)
    axes[1, idx].set_title(f'Sil={sil:.3f}, ARI={ari:.3f}')
    axes[1, idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"{'Metodo':<10} {'Silhouette':>12} {'ARI':>10}")
for m in metodi:
    mark = '*' if m == max(risultati, key=lambda x: risultati[x]['sil']) else ' '
    print(f"{mark}{m:<9} {risultati[m]['sil']:>12.3f} {risultati[m]['ari']:>10.3f}")

best = max(risultati, key=lambda x: risultati[x]['sil'])
worse = min(risultati, key=lambda x: risultati[x]['sil'])
print(f"Metodo migliore: {best.upper()} (silhouette {risultati[best]['sil']:.3f})")
print(f"Metodo peggiore: {worse.upper()} (silhouette {risultati[worse]['sil']:.3f})")


### Perche questo esercizio (22.3)
Applicare AgglomerativeClustering di sklearn a clienti e-commerce testando diversi K e rinominando i cluster in modo interpretabile.


In [ ]:
# Esercizio 22.3 - AgglomerativeClustering su clienti e-commerce
# Obiettivo: testare K diversi, scegliere quello con silhouette migliore e nominare i cluster.

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering

spesa_media = [50, 55, 48, 200, 250, 230, 120, 130, 125, 45, 280, 115]
frequenza_acquisti = [2, 3, 2, 8, 10, 9, 5, 6, 5, 2, 12, 4]

df_clienti = pd.DataFrame({
    'cliente': [f'C{i}' for i in range(1, 13)],
    'spesa_media': spesa_media,
    'frequenza_acquisti': frequenza_acquisti
})
print("Dataset clienti (describe):")
print(df_clienti[['spesa_media','frequenza_acquisti']].describe().round(1))

X_clienti = df_clienti[['spesa_media', 'frequenza_acquisti']].values
X_clienti_scaled = StandardScaler().fit_transform(X_clienti)
assert X_clienti_scaled.shape[0] == len(df_clienti)

K_range = [2, 3, 4]
risultati = {}
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, k in enumerate(K_range):
    agg = AgglomerativeClustering(n_clusters=k, linkage='ward')
    labels = agg.fit_predict(X_clienti_scaled)
    sil = silhouette_score(X_clienti_scaled, labels)
    risultati[k] = {'sil': sil, 'labels': labels}
    axes[idx].scatter(df_clienti['spesa_media'], df_clienti['frequenza_acquisti'], c=labels, cmap='viridis', s=100, edgecolors='black', alpha=0.8)
    for i, row in df_clienti.iterrows():
        axes[idx].annotate(row['cliente'], (row['spesa_media']+3, row['frequenza_acquisti']+0.1), fontsize=8)
    axes[idx].set_title(f'K={k}, silhouette={sil:.3f}')
    axes[idx].set_xlabel('Spesa media (EUR)')
    axes[idx].set_ylabel('Frequenza acquisti')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

K_ottimale = max(risultati, key=lambda x: risultati[x]['sil'])
print(f"K ottimale: {K_ottimale} (silhouette {risultati[K_ottimale]['sil']:.3f})")

df_clienti['cluster'] = risultati[K_ottimale]['labels']
profili = df_clienti.groupby('cluster')[['spesa_media','frequenza_acquisti']].mean().round(1)
print("Profili medi per cluster:")
print(profili)

nomi_cluster = {}
for cluster_id, row in profili.iterrows():
    if row['spesa_media'] > 180 and row['frequenza_acquisti'] > 7:
        nome = 'Premium alta spesa'
    elif row['spesa_media'] > 90 and row['frequenza_acquisti'] > 4:
        nome = 'Clienti fedeli'
    else:
        nome = 'Occasionali a bassa spesa'
    nomi_cluster[cluster_id] = nome

print("Nomi assegnati:")
for cid, nome in nomi_cluster.items():
    clienti = ', '.join(df_clienti[df_clienti['cluster'] == cid]['cliente'].values)
    print(f"Cluster {cid}: {nome} -> {clienti}")

fig, ax = plt.subplots(figsize=(8, 6))
colors = plt.cm.viridis(np.linspace(0, 1, len(profili)))
for cid in profili.index:
    mask = df_clienti['cluster'] == cid
    ax.scatter(df_clienti.loc[mask, 'spesa_media'], df_clienti.loc[mask, 'frequenza_acquisti'], c=[colors[cid]], s=140, edgecolors='black', alpha=0.85, label=nomi_cluster[cid])
for i, row in df_clienti.iterrows():
    ax.annotate(row['cliente'], (row['spesa_media']+3, row['frequenza_acquisti']+0.1), fontsize=8)
ax.set_xlabel('Spesa media (EUR)')
ax.set_ylabel('Frequenza acquisti')
ax.set_title(f'AgglomerativeClustering (K={K_ottimale})')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


## Sezione 6 - Conclusione operativa

---

## I 5 Take-Home Messages

### 1. Il dendrogramma mostra TUTTE le fusioni
A differenza di K-Means che richiede K a priori, il gerarchico costruisce l'intero albero e ti lascia scegliere dove tagliare.

```
Distanza
   │
   │    ╔═══════════╗  ← Taglio alto = 2 cluster
   │    ║           ║
   │   ╔╝           ╚╗
   │   ║             ║  ← Taglio medio = 3 cluster
   │  ╔╝             ╚╗
   │  ║               ║
   │ ╔╝               ╚╗ ← Taglio basso = 5 cluster
   └──────────────────────
```

### 2. Ward è il default, Single è pericoloso

| Linkage | Quando usarlo |
|---------|---------------|
| **Ward** | Default. Cluster bilanciati, simili a K-Means |
| Complete | Vuoi cluster compatti, senza outlier |
| Average | Compromesso, dati rumorosi |
| Single | MAI (effetto catena), solo per forme molto allungate |

**Effetto catena:** Single collega cluster lontani attraverso punti "ponte", creando un unico cluster allungato.

### 3. Leggi le "gambe lunghe" nel dendrogramma
- **Gambe lunghe** = grande salto di distanza = separazione naturale
- **Taglia sotto una gamba lunga** per ottenere cluster ben separati
- Se non ci sono gambe lunghe evidenti, i dati potrebbero non avere struttura cluster

### 4. Usa silhouette per validare il taglio
```python
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import silhouette_score

# Prova diversi tagli
for n_clusters in [2, 3, 4, 5]:
    labels = fcluster(Z, n_clusters, criterion='maxclust')
    sil = silhouette_score(X_scaled, labels)
    print(f"K={n_clusters}: Silhouette={sil:.3f}")
```

### 5. Gerarchico vs K-Means: tabella decisionale

| Criterio | K-Means | Gerarchico |
|----------|---------|------------|
| Dataset size | Qualsiasi | < 10.000 punti |
| K noto | ✅ Sì | ❌ No, lo scopri |
| Velocità | O(n·K·iter) | O(n²) o O(n³) |
| Determinismo | ❌ Random init | ✅ Sempre uguale |
| Output | Solo etichette | Albero completo |
| Forma cluster | Sferica | Dipende da linkage |

---

## Template completo clustering gerarchico

```python
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# 1. PREPROCESSING
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. CALCOLA LINKAGE
Z = linkage(X_scaled, method='ward')

# 3. VISUALIZZA DENDROGRAMMA
plt.figure(figsize=(10, 5))
dendrogram(Z, truncate_mode='lastp', p=30)
plt.xlabel('Campioni')
plt.ylabel('Distanza')
plt.title('Dendrogramma (Ward linkage)')
plt.axhline(y=threshold, color='r', linestyle='--', label=f'Taglio a {threshold}')
plt.show()

# 4. ESTRAI CLUSTER
n_clusters = 3  # scelto dal dendrogramma
labels = fcluster(Z, n_clusters, criterion='maxclust')

# 5. VALUTA
print(f"Silhouette: {silhouette_score(X_scaled, labels):.3f}")
```

---

## Flowchart decisionale

```
               DATI SCALATI
                    │
                    ▼
            CALCOLA linkage(X, 'ward')
                    │
                    ▼
            VISUALIZZA dendrogramma
                    │
        ┌───────────┴───────────┐
        │                       │
   Gambe lunghe?           No gambe?
        │                       │
        ▼                       ▼
   Taglia sotto           Nessuna struttura
        │                  → prova altri linkage
        ▼                  → o dati senza cluster
   fcluster(Z, K)
        │
        ▼
   Valida con silhouette
```

---

## Prossimi passi
→ **Lezione 23**: DBSCAN - clustering basato su densità per forme arbitrarie

## Sezione 7 - End-of-lesson checklist e glossario

### Checklist finale
- [ ] Dati numerici scalati con `StandardScaler`
- [ ] Linkage calcolato con metodo motivato (Ward default)
- [ ] Dendrogramma visualizzato e letto (gambe lunghe, salti)
- [ ] Altezza di taglio scelta con criterio
- [ ] Cluster estratti con `fcluster` o `AgglomerativeClustering`
- [ ] Silhouette calcolata per validare la scelta
- [ ] Cluster interpretati e rinominati con etichette leggibili
- [ ] Confronto rapido con K-Means se utile

---

### Glossario (termini usati)

| # | Termine | Definizione |
|---|---------|-------------|
| 1 | Agglomerativo | Fusione bottom-up: ogni punto → cluster → un unico cluster |
| 2 | Divisivo | Divisione top-down (non in sklearn) |
| 3 | Dendrogramma | Albero delle fusioni, distanza sull'asse Y |
| 4 | Linkage | Criterio di distanza tra cluster |
| 5 | Ward | Linkage che minimizza l'incremento di varianza |
| 6 | Single | Distanza minima tra punti (effetto catena!) |
| 7 | Complete | Distanza massima tra punti |
| 8 | Average | Media di tutte le distanze |
| 9 | Effetto catena | Problema di Single: cluster fusi via punti ponte |
| 10 | Taglio | Linea orizzontale che determina K |
| 11 | fcluster | Funzione scipy per estrarre etichette |
| 12 | Gambe lunghe | Salti di distanza nel dendrogramma → separazioni naturali |

---

## Sezione 8 - Didactic changelog

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | Originale | Versione iniziale del notebook |
| 2.0 | 2026-01-XX | Riorganizzata nelle 8 sezioni obbligatorie |
| 2.1 | 2026-01-XX | Aggiunte rationale e checkpoint a demo e esercizi |
| 2.2 | 2026-01-XX | Inserite Methods explained, Common errors, Glossario |
| 2.3 | 2026-01-XX | **Espansione didattica completa**: mappa lezione con tempi; ASCII visualization 4 linkage; confronto K-Means vs gerarchico; ASCII dendrogramma con livelli taglio; 5 take-home messages; template Python completo; flowchart decisionale; tabella linkage con raccomandazioni. |

---

## Note di rilascio v2.3

### Contenuti aggiunti
- **Header**: mappa temporale, ASCII linkage, tabella confronto
- **Conclusione**: 5 principi, ASCII dendrogramma, template Python, flowchart

### Miglioramenti pedagogici
- Visualizzazione ASCII dei 4 metodi di linkage
- Warning esplicito su Single linkage e effetto catena
- Regola "gambe lunghe" con interpretazione
- Tabella decisionale gerarchico vs K-Means

### Competenze verificabili
Dopo questa lezione lo studente può:
1. Costruire un dendrogramma con scipy
2. Scegliere il linkage appropriato
3. Tagliare e validare con silhouette
4. Decidere tra gerarchico e K-Means

---

**Fine della lezione**